<img width="400" src="https://nomeroff.net.ua/images/logo/nomeroff_net.svg" alt="Nomeroff Net. Automatic numberplate recognition system" align="left"/>

## Quick recognition example.

Before runing this demo, please download models from [https://nomeroff.net.ua/models/](https://nomeroff.net.ua/models/) to **./models/** directory. 

In [1]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
# os.environ["CUDA_VISIBLE_DEVICES"] = ""  # For CPU inference

In [2]:
# dynamically grow the memory used on the GPU
from tensorflow.compat.v1.keras.backend import set_session
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True 
sess = tf.compat.v1.Session(config=config)
set_session(sess)

In [3]:
# Import all necessary libraries.
import os
import numpy as np
import sys
import matplotlib.image as mpimg

# NomeroffNet path
NOMEROFF_NET_DIR = os.path.abspath('../')
sys.path.append(NOMEROFF_NET_DIR)

In [4]:
# Import license plate recognition tools.
from NomeroffNet import  Detector
from NomeroffNet import  filters
from NomeroffNet import  RectDetector
from NomeroffNet import  OptionsDetector
from NomeroffNet import  TextDetector
from NomeroffNet import  textPostprocessing

# load models
nnet = Detector()
nnet.loadModel(NOMEROFF_NET_DIR)

rectDetector = RectDetector()

optionsDetector = OptionsDetector()
optionsDetector.load_pb("../models/trt_saved_model_options")

textDetector = TextDetector.get_static_module("eu")()
textDetector.load_pb("../models/trt_saved_model_ocr_eu")

Config '/var/www/nomeroff-net/./NomeroffNet/configs/centermask2/numberplates/centermask_numberplate_V_39_eSE_FPN_ms_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.


In [5]:
# Detect numberplate
img_path = 'images/example2.jpeg'
img = mpimg.imread(img_path)
NP = nnet.detect_mask([img])

/var/www/nomeroff-net/centermask2/centermask/modeling/fcos/fcos_outputs.py:402: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  per_candidate_nonzeros = per_candidate_inds.nonzero()


In [7]:
%%time
# Generate image mask.
cv_imgs_masks = nnet.detect_mask([img])

for i in range(100):
    for cv_img_masks in cv_imgs_masks:
        # Detect points.
        arrPoints = rectDetector.detect(cv_img_masks)

        # cut zones
        zones = rectDetector.get_cv_zonesBGR(img, arrPoints, 64, 295)

        # find standart
        regionIds, stateIds, countLines = optionsDetector.predict_pb(zones)
        regionNames = optionsDetector.getRegionLabels(regionIds)

        # find text with postprocessing by standart  
        textArr = textDetector.predict_pb(zones)
        textArr = textPostprocessing(textArr, regionNames)

CPU times: user 58.4 s, sys: 34.5 s, total: 1min 32s
Wall time: 8.11 s


## keras result

CPU times: user 59.7 s, sys: 35.3 s, total: 1min 35s
Wall time: 9.64 s
 
CPU times: user 1min, sys: 35.2 s, total: 1min 35s
Wall time: 9.64 s

CPU times: user 1min, sys: 35.2 s, total: 1min 35s
Wall time: 9.76 s

## trt result